In [ ]:
import os
import sys

# 设置项目根目录
project_root = "/home/wjw/lerobot"
sys.path.insert(0, project_root)
os.chdir(project_root)

print("📂 当前工作目录:", os.getcwd())


In [ ]:
print("🔍 当前可用的 /dev/ttyACM* 端口:")
!ls /dev/ttyACM*

print("\n🔍 查询 /dev/serial/by-id/ 绑定:")
!ls -l /dev/serial/by-id/

# 用户手动确认端口信息
input("🔴 请确认上面信息与 `configs.py` 中的端口匹配，若不匹配，请手动更新后按 Enter 继续...")


In [ ]:
import shutil

calib_path = "/home/wjw/lerobot/.cache/calibration/koch/"
shutil.rmtree(calib_path, ignore_errors=True)

print(f"🧹  已删除旧的标定文件: {calib_path}")


In [ ]:
from lerobot.common.robot_devices.robots.configs import KochRobotConfig
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot

# 实例化 Koch 配置
robot_cfg = KochRobotConfig()
robot_cfg.cameras = {}  # 关闭摄像头，专注于机械臂

# 连接机械臂
robot = ManipulatorRobot(robot_cfg)
print("✅  机械臂初始化完成")

# 进入标定
robot.connect()
print("📌  机械臂已连接，标定文件将保存至:", robot_cfg.calibration_dir)


In [ ]:
print("📁 标定文件内容:")
!ls -l /home/wjw/lerobot/.cache/calibration/koch/


In [ ]:
import time
import tqdm

print("🎮 开始主从同步测试，时长 30 秒，频率 200Hz...")
seconds = 30
frequency = 200
interval = 1 / frequency

for _ in tqdm.tqdm(range(seconds * frequency)):
    leader_pos = robot.leader_arms["main"].read("Present_Position")
    robot.follower_arms["main"].write("Goal_Position", leader_pos)
    time.sleep(interval)

print("✅  主从臂同步测试完成！")


In [ ]:
# 读取主臂状态
leader_status = robot.leader_arms["main"].read("Hardware_Error_Status")
leader_torque = robot.leader_arms["main"].read("Torque_Enable")
leader_position = robot.leader_arms["main"].read("Present_Position")

# 读取从臂状态
follower_status = robot.follower_arms["main"].read("Hardware_Error_Status")
follower_torque = robot.follower_arms["main"].read("Torque_Enable")
follower_position = robot.follower_arms["main"].read("Present_Position")

# 输出状态信息
print("✅ 主臂硬件错误状态:", leader_status)
print("✅ 主臂扭矩状态:", leader_torque)
print("✅ 主臂当前关节位置:", leader_position)
print("✅ 从臂硬件错误状态:", follower_status)
print("✅ 从臂扭矩状态:", follower_torque)
print("✅ 从臂当前关节位置:", follower_position)


In [ ]:
from lerobot.common.robot_devices.motors.dynamixel import TorqueMode

# 关闭主从臂扭矩
robot.leader_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)
robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)

# 确保扭矩关闭
leader_torque = robot.leader_arms["main"].read("Torque_Enable")
follower_torque = robot.follower_arms["main"].read("Torque_Enable")

assert all(t == 0 for t in leader_torque), "❌ 主臂扭矩未正确关闭！"
assert all(t == 0 for t in follower_torque), "❌ 从臂扭矩未正确关闭！"

print("✅  主从臂扭矩已确认关闭！")



In [ ]:
# 断开机械臂连接
robot.disconnect()
print("✅  机械臂已断开连接！")


In [1]:
根据教程的内容，你已经完成了 **机械臂的标定（Calibration）** 和 **主从同步遥控（Teleoperation）**，接下来的步骤包括：

### **📌 下一步要做的事情**
1. **数据采集（Recording Dataset）**
   - 你需要录制主从遥控的数据，记录机械臂的运动轨迹，生成数据集用于训练神经网络。
   - 目标是让神经网络学习人类演示的动作，从而实现自动化控制。

2. **训练神经网络（Training a Policy）**
   - 使用数据集训练模仿学习模型，比如 **ACT (Action Chunking Transformer)**，让机器人学习如何执行任务。
   - 训练后，你可以让模型在真实机器人上进行推理。

3. **评估和优化模型（Evaluating the Model）**
   - 你可以让训练好的神经网络控制机器人，观察其执行任务的情况，并调整模型参数优化效果。

4. **可视化数据集和回放（Visualizing Dataset & Replay）**
   - 你可以回放已经录制的数据，检查数据质量，并确保采集的数据可用于训练。

---

### **🚀 如何进行下一步？**
#### **📌 1. 录制数据集**
可以使用 `record` 函数来录制机器人动作数据：

#### **👉 运行以下命令来录制数据**
```bash
python lerobot/scripts/control_robot.py \
  --robot.type=koch \
  --control.type=record \
  --control.single_task="Grasp a lego block and put it in the bin." \
  --control.fps=30 \
  --control.repo_id=${HF_USER}/koch_test \
  --control.tags='["tutorial"]' \
  --control.warmup_time_s=5 \
  --control.episode_time_s=30 \
  --control.reset_time_s=30 \
  --control.num_episodes=2 \
  --control.push_to_hub=true
```
**解释：**
- `--control.single_task="Grasp a lego block and put it in the bin."` 设定任务名称
- `--control.fps=30` 设置采集帧率为 30Hz
- `--control.num_episodes=2` 录制 2 个演示数据
- `--control.push_to_hub=true` 录制的数据将自动上传到 Hugging Face 服务器（如果已登录）

你可以把 `num_episodes` 调大，比如 `50`，录制更多数据。

---

#### **📌 2. 训练模仿学习模型**
使用 `train.py` 脚本训练一个 **ACT（Action Chunking Transformer）** 模型，让机器人学习如何执行任务。

```bash
python lerobot/scripts/train.py \
  --dataset.repo_id=${HF_USER}/koch_test \
  --policy.type=act \
  --output_dir=outputs/train/act_koch_test \
  --job_name=act_koch_test \
  --policy.device=cuda \
  --wandb.enable=true
```
**解释：**
- `--dataset.repo_id=${HF_USER}/koch_test` 指定数据集来源
- `--policy.type=act` 选择 **ACT** 模型
- `--policy.device=cuda` 在 **GPU** 运行（如果没有 GPU，可以改成 `cpu`）

---

#### **📌 3. 让机器人自动执行任务**
训练完成后，你可以加载模型，让机器人自动执行任务：
```bash
python lerobot/scripts/control_robot.py \
  --robot.type=koch \
  --control.type=record \
  --control.fps=30 \
  --control.repo_id=${HF_USER}/eval_act_koch_test \
  --control.tags='["tutorial"]' \
  --control.warmup_time_s=5 \
  --control.episode_time_s=30 \
  --control.reset_time_s=30 \
  --control.num_episodes=10 \
  --control.push_to_hub=true \
  --control.policy.path=outputs/train/act_koch_test/checkpoints/last/pretrained_model
```
**解释：**
- `--control.policy.path=outputs/train/act_koch_test/checkpoints/last/pretrained_model` 载入训练好的模型
- `--control.num_episodes=10` 评估 10 轮

---

### **🎯 结论**
✅ 你现在可以：
1. 录制数据集（**record**）
2. 训练模仿学习模型（**train**）
3. 让机器人自动执行任务（**evaluate**）

下一步建议 **先录制数据集**，确保数据足够后再进行训练 🚀。如果遇到问题，欢迎继续交流！

SyntaxError: invalid character '，' (U+FF0C) (3888031187.py, line 1)